# Continuous Training with AutoML Vertex Pipelines with Batch Predictions

**Learning Objectives:**
1. Learn how to use Vertex AutoML pre-built components
1. Learn how to build a Vertex AutoML pipeline with these components using BigQuery as a data source
1. Learn how to compile, upload, and run the Vertex AutoML pipeline
1. Serve batch predictions with BigQuery source from the AutoML pipeline


In this lab, you will build, deploy, and run a Vertex AutoML pipeline that orchestrates the **Vertex AutoML AI** services to train, tune, and serve batch predictions to BigQuery with a model. 

## Setup

In [1]:
import os

from google.cloud import aiplatform

In [2]:
REGION = "us-central1"
PROJECT = !(gcloud config get-value project)
PROJECT = PROJECT[0]

os.environ["PROJECT"] = PROJECT

In [3]:
# Set `PATH` to include the directory containing KFP CLI
PATH = %env PATH
%env PATH=/home/jupyter/.local/bin:{PATH}

env: PATH=/home/jupyter/.local/bin:/usr/local/cuda/bin:/opt/conda/bin:/opt/conda/condabin:/usr/local/bin:/usr/bin:/bin:/usr/local/games:/usr/games


### BigQuery Data

If you have not gone through the KFP Walkthrough lab, you will need to run the following cell to create a BigQuery dataset and table containing the data required for this lab.

**NOTE** If you already have the covertype data in a bigquery table at `<PROJECT_ID>.covertype_dataset.covertype` you may skip to **Understanding the pipeline design**.

In [4]:
%%bash

DATASET_LOCATION=US
DATASET_ID=covertype_dataset
TABLE_ID=covertype
DATA_SOURCE=gs://workshop-datasets/covertype/small/dataset.csv
SCHEMA=Elevation:INTEGER,\
Aspect:INTEGER,\
Slope:INTEGER,\
Horizontal_Distance_To_Hydrology:INTEGER,\
Vertical_Distance_To_Hydrology:INTEGER,\
Horizontal_Distance_To_Roadways:INTEGER,\
Hillshade_9am:INTEGER,\
Hillshade_Noon:INTEGER,\
Hillshade_3pm:INTEGER,\
Horizontal_Distance_To_Fire_Points:INTEGER,\
Wilderness_Area:STRING,\
Soil_Type:STRING,\
Cover_Type:INTEGER

bq --location=$DATASET_LOCATION --project_id=$PROJECT mk --dataset $DATASET_ID

bq --project_id=$PROJECT --dataset_id=$DATASET_ID load \
--source_format=CSV \
--skip_leading_rows=1 \
--replace \
$TABLE_ID \
$DATA_SOURCE \
$SCHEMA

BigQuery error in mk operation: Dataset 'kylesteckler-demo:covertype_dataset'
already exists.


Waiting on bqjob_r21048939c4d82a5f_000001809057d82b_1 ... (3s) Current status: DONE   


## Understanding the pipeline design


The workflow implemented by the pipeline is defined using a Python based Domain Specific Language (DSL). The pipeline's DSL is in the `pipeline_vertex/pipeline_vertex_automl_batch_preds.py` file that we will generate below.

The pipeline's DSL has been designed to avoid hardcoding any environment specific settings like file paths or connection strings. These settings are provided to the pipeline code through a set of environment variables.


## Building and deploying the pipeline

Let us write the pipeline to disk:

In [5]:
%%writefile ./pipeline_vertex/pipeline_vertex_automl_batch_preds.py
"""Kubeflow Covertype Pipeline."""

import os

from google_cloud_pipeline_components.aiplatform import (
    AutoMLTabularTrainingJobRunOp,
    TabularDatasetCreateOp,
    ModelBatchPredictOp
)
from kfp.v2 import dsl

PIPELINE_ROOT = os.getenv("PIPELINE_ROOT")
PROJECT = os.getenv("PROJECT")
DATASET_SOURCE = os.getenv("DATASET_SOURCE")
PIPELINE_NAME = os.getenv("PIPELINE_NAME", "covertype")
DISPLAY_NAME = os.getenv("MODEL_DISPLAY_NAME", PIPELINE_NAME)
TARGET_COLUMN = os.getenv("TARGET_COLUMN", "Cover_Type")
BATCH_PREDS_SOURCE_URI = os.getenv("BATCH_PREDS_SOURCE_URI")

@dsl.pipeline(
    name=f"{PIPELINE_NAME}-vertex-automl-pipeline-batch-preds",
    description=f"AutoML Vertex Pipeline for {PIPELINE_NAME}",
    pipeline_root=PIPELINE_ROOT,
)
def create_pipeline():

    dataset_create_task = TabularDatasetCreateOp(
        display_name=DISPLAY_NAME,
        bq_source=DATASET_SOURCE,
        project=PROJECT,
    )

    automl_training_task = AutoMLTabularTrainingJobRunOp(
        project=PROJECT,
        display_name=DISPLAY_NAME,
        optimization_prediction_type="classification",
        dataset=dataset_create_task.outputs["dataset"],
        target_column=TARGET_COLUMN,
    )

    batch_predict_op = ModelBatchPredictOp(
        project=PROJECT,
        job_display_name="batch_predict_job",
        model=automl_training_task.outputs["model"],
        bigquery_source_input_uri=BATCH_PREDS_SOURCE_URI,
        instances_format="bigquery",
        predictions_format="bigquery",
        bigquery_destination_output_uri=f'bq://{PROJECT}',
    )


Overwriting ./pipeline_vertex/pipeline_vertex_automl_batch_preds.py


### Create BigQuery table with new data
Before we compile and run the pipeline, let's create a BigQuery table with data we want it to serve batch predictions on. To simulate "new" data we will simply query the existing table for all columns except the label.

In [6]:
%%bigquery
CREATE OR REPLACE TABLE covertype_dataset.newdata AS 
SELECT * EXCEPT(Cover_Type)
FROM covertype_dataset.covertype
LIMIT 10000

Query complete after 0.01s: 100%|██████████| 3/3 [00:00<00:00, 1720.86query/s]                        


""


### Compile the pipeline

Let's start by defining the environment variables that will be passed to the pipeline compiler:

In [7]:
ARTIFACT_STORE = f"gs://{PROJECT}-kfp-artifact-store"
PIPELINE_ROOT = f"{ARTIFACT_STORE}/pipeline"
DATASET_SOURCE = f"bq://{PROJECT}.covertype_dataset.covertype"
BATCH_PREDS_SOURCE_URI = f"bq://{PROJECT}.covertype_dataset.newdata"

%env PIPELINE_ROOT={PIPELINE_ROOT}
%env PROJECT={PROJECT}
%env REGION={REGION}
%env DATASET_SOURCE={DATASET_SOURCE}
%env BATCH_PREDS_SOURCE_URI={BATCH_PREDS_SOURCE_URI}

env: PIPELINE_ROOT=gs://kylesteckler-demo-kfp-artifact-store/pipeline
env: PROJECT=kylesteckler-demo
env: REGION=us-central1
env: DATASET_SOURCE=bq://kylesteckler-demo.covertype_dataset.covertype
env: BATCH_PREDS_SOURCE_URI=bq://kylesteckler-demo.covertype_dataset.newdata


Let us make sure that the `ARTIFACT_STORE` has been created, and let us create it if not:

In [8]:
!gsutil ls | grep ^{ARTIFACT_STORE}/$ || gsutil mb -l {REGION} {ARTIFACT_STORE}

gs://kylesteckler-demo-kfp-artifact-store/


#### Use the CLI compiler to compile the pipeline

We compile the pipeline from the Python file we generated into a JSON description using the following command:

In [9]:
PIPELINE_JSON = "covertype_automl_vertex_pipeline_batch_preds.json"

In [10]:
!dsl-compile-v2 --py pipeline_vertex/pipeline_vertex_automl_batch_preds.py --output $PIPELINE_JSON

/opt/conda/lib/python3.7/site-packages/kfp/v2/compiler/compiler.py:1266: FutureWarning: APIs imported from the v1 namespace (e.g. kfp.dsl, kfp.components, etc) will not be supported by the v2 compiler since v2.0.0
  category=FutureWarning,


**Note:** You can also use the Python SDK to compile the pipeline:

```python
from kfp.v2 import compiler

compiler.Compiler().compile(
    pipeline_func=create_pipeline, 
    package_path=PIPELINE_JSON,
)

```

The result is the pipeline file. 

In [11]:
!head {PIPELINE_JSON}

{
  "pipelineSpec": {
    "components": {
      "comp-automltabulartrainingjob-run": {
        "executorLabel": "exec-automltabulartrainingjob-run",
        "inputDefinitions": {
          "artifacts": {
            "dataset": {
              "artifactType": {
                "schemaTitle": "google.VertexDataset",


### Deploy the pipeline package

In [12]:
aiplatform.init(project=PROJECT, location=REGION)

pipeline = aiplatform.PipelineJob(
    display_name="automl_covertype_kfp_pipeline",
    template_path=PIPELINE_JSON,
    enable_caching=True,
)

pipeline.run()

Creating PipelineJob
PipelineJob created. Resource name: projects/693210680039/locations/us-central1/pipelineJobs/covertype-vertex-automl-pipeline-batch-preds-20220504183214
To use this PipelineJob in another session:
pipeline_job = aiplatform.PipelineJob.get('projects/693210680039/locations/us-central1/pipelineJobs/covertype-vertex-automl-pipeline-batch-preds-20220504183214')
View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/covertype-vertex-automl-pipeline-batch-preds-20220504183214?project=693210680039
PipelineJob run completed. Resource name: projects/693210680039/locations/us-central1/pipelineJobs/covertype-vertex-automl-pipeline-batch-preds-20220504183214


### Understanding the resources created by BatchPredictOp

Once the pipeline has finished running you will see a new BigQuery dataset with name `prediction_<model-display-name>_<job-create-time>`. Inside this dataset you will see a `predictions` table, containing the batch prediction examples and predicted labels. If there were any errors in the batch prediction, you will also see an `errors` table. The errors table contains rows for which the prediction has failed.

Copyright 2021 Google LLC

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.